In [1]:
import json
import numpy as np
from torchvision import datasets, transforms
from kafka import KafkaProducer, KafkaConsumer
import redis
import time

# Redis setup
redis_client = redis.Redis(host='host.docker.internal', port=6379, db=0)

# Kafka setup
producer = KafkaProducer(bootstrap_servers='kafka:29092',
                         value_serializer=lambda v: json.dumps(v).encode('utf-8'))
consumer = KafkaConsumer('activate-layer', bootstrap_servers='kafka:29092',
                         value_deserializer=lambda m: json.loads(m.decode('utf-8')))

# Load JSON file
def load_network(filename):
    with open(filename, 'r') as f:
        return json.load(f)

# Activation functions
def relu(x):
    return np.maximum(0, x)

def softmax(x):
    e_x = np.exp(x - np.max(x))  # Stability trick
    return e_x / e_x.sum()

ACTIVATIONS = {
    "relu": relu,
    "softmax": softmax
}

class Neuron:
    def __init__(self, weights, bias, activation, is_final_layer=False):
        self.weights = np.array(weights)
        self.bias = np.array(bias)
        self.activation_func = None if is_final_layer else ACTIVATIONS.get(activation, relu)
        self.is_final_layer = is_final_layer

    def forward(self, inputs):
        """Compute neuron output"""
        z = np.dot(inputs, self.weights) + self.bias  # (num_features,)
        return z if self.is_final_layer else self.activation_func(z)

class Layer:
    def __init__(self, layer_id, neurons, is_final_layer=False):
        self.layer_id = layer_id
        self.neurons = neurons
        self.is_final_layer = is_final_layer

    def forward(self, input_data):
        """Process one input sample at a time"""
        outputs = np.array([neuron.forward(input_data) for neuron in self.neurons])
        if self.is_final_layer:
            outputs = softmax(outputs)
        redis_client.set(self.layer_id, outputs.astype(np.float32).tobytes())
        producer.send('activate-layer', {'layer': self.layer_id})
        return outputs

# Build network
def build_network(json_data):
    layers = []
    sorted_layers = sorted(json_data.keys(), key=lambda x: int(x.split('_')[-1]))
    for i, layer_name in enumerate(sorted_layers):
        layer_info = json_data[layer_name]
        neurons = [
            Neuron(
                weights=np.array(node['weights']),
                bias=np.array(node['biases']),
                activation=node['activation'],
                is_final_layer=(i == len(sorted_layers) - 1)
            )
            for node in layer_info['nodes']
        ]
        layers.append(Layer(layer_id=layer_name, neurons=neurons, is_final_layer=(i == len(sorted_layers) - 1)))
    return layers

# Forward pass for single image
def forward_pass(layers, input_data):
    for layer in layers:
        input_data = layer.forward(input_data)
    redis_client.set('results', json.dumps(int(np.argmax(input_data))))
    producer.send('activate-layer', {'layer': 'final'})

# Load network
data = load_network("node_based_model.json")
network = build_network(data)

# Load MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])
mnist_test = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

# Test first 10 images one by one
correct_predictions = 0
for i in range(10):
    image, label = mnist_test[i]
    image_np = image.view(-1).numpy()
    redis_client.delete('results')
    forward_pass(network, image_np)

    timeout = 10  # seconds
    waited = 0
    while redis_client.get('results') is None and waited < timeout:
        time.sleep(0.1)
        waited += 0.1

    prediction_json = redis_client.get('results')
    if prediction_json:
        prediction = json.loads(prediction_json)
        print(f"Image {i} Prediction: {prediction}, Label: {label}")
        if prediction == label:
            correct_predictions += 1
    else:
        print(f"No prediction received for image {i} within timeout.")

accuracy = correct_predictions / 10
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Image 0 Prediction: 7, Label: 7
Image 1 Prediction: 2, Label: 2
Image 2 Prediction: 1, Label: 1
Image 3 Prediction: 0, Label: 0
Image 4 Prediction: 4, Label: 4
Image 5 Prediction: 1, Label: 1
Image 6 Prediction: 4, Label: 4
Image 7 Prediction: 9, Label: 9
Image 8 Prediction: 6, Label: 5
Image 9 Prediction: 9, Label: 9
Test Accuracy: 90.00%


In [2]:
import json
import numpy as np
from torchvision import datasets, transforms
from kafka import KafkaProducer, KafkaConsumer
import redis
import time

# Redis setup
redis_client = redis.Redis(host='host.docker.internal', port=6379, db=0)

# Activation functions
def relu(x):
    return np.maximum(0, x)

def softmax(x):
    e_x = np.exp(x - np.max(x))  # Stability trick
    return e_x / e_x.sum()

ACTIVATIONS = {
    "relu": relu,
    "softmax": softmax
}

class Neuron:
    def __init__(self, layer_id, neuron_id, weights, bias, activation, is_final_layer=False):
        self.layer_id = layer_id
        self.neuron_id = neuron_id
        self.weights = np.array(weights)
        self.bias = np.array(bias)
        self.activation_func = None if is_final_layer else ACTIVATIONS.get(activation, relu)
        self.is_final_layer = is_final_layer

    def forward(self, inputs):
        """Wait for activation message from Kafka before processing"""
        topic = f'layer-{self.layer_id[-1]}'
        consumer = KafkaConsumer(topic, bootstrap_servers='kafka:9092',
                                 value_deserializer=lambda m: json.loads(m.decode('utf-8')), auto_offset_reset='latest',enable_auto_commit=True)
        print(f"🕒 Neuron {self.neuron_id} in {self.layer_id} waiting for activation on topic {topic}...")

        for message in consumer:
            print(f"✅ Neuron {self.neuron_id} activated with message: {message.value}")
            if message.value['layer'] == self.layer_id:
                break  # Process once activation message is received
        consumer.close()

        z = np.dot(inputs, self.weights) + self.bias
        return z if self.is_final_layer else self.activation_func(z)

class Layer:
    def __init__(self, layer_id, neurons, is_final_layer=False):
        self.layer_id = layer_id
        self.neurons = neurons
        self.is_final_layer = is_final_layer

    def forward(self, input_data):
        """Trigger neuron activations via Kafka"""
        producer = KafkaProducer(bootstrap_servers='kafka:9092',
                                 value_serializer=lambda v: json.dumps(v).encode('utf-8'),
                                 retries=5, request_timeout_ms=10000)
    
        topic = f'layer-{self.layer_id[-1]}'  # Ensure topic follows `layer-0`, `layer-1` format
        activation_message = {'layer': self.layer_id}
    
        print(f"📤 Layer {self.layer_id} sending activation message to topic {topic}...")
    
        try:
            future = producer.send(topic, activation_message)
            result = future.get(timeout=10)  # Wait for response
            print(f"✅ Kafka message sent to {topic}: {result}")
        except Exception as e:
            print(f"❌ Kafka send failed: {e}")
    
        producer.flush()
        producer.close()
    
        outputs = np.array([neuron.forward(input_data) for neuron in self.neurons])
        if self.is_final_layer:
            outputs = softmax(outputs)
    
        redis_client.set(self.layer_id, outputs.astype(np.float32).tobytes())
        return outputs


# Load JSON file
def load_network(filename):
    with open(filename, 'r') as f:
        return json.load(f)

# Build network
def build_network(json_data):
    layers = []
    sorted_layers = sorted(json_data.keys(), key=lambda x: int(x.split('_')[-1]))
    for i, layer_name in enumerate(sorted_layers):
        layer_info = json_data[layer_name]
        neurons = [
            Neuron(
                layer_id=layer_name,
                neuron_id=idx,
                weights=np.array(node['weights']),
                bias=np.array(node['biases']),
                activation=node['activation'],
                is_final_layer=(i == len(sorted_layers) - 1)
            )
            for idx, node in enumerate(layer_info['nodes'])
        ]
        layers.append(Layer(layer_id=layer_name, neurons=neurons, is_final_layer=(i == len(sorted_layers) - 1)))
    return layers

# Forward pass for single image
def forward_pass(layers, input_data):
    producer = KafkaProducer(bootstrap_servers='kafka:9092',
                             value_serializer=lambda v: json.dumps(v).encode('utf-8'),
                            )
    print("🔥 Sending initial activation message to layer-0...")
    producer.send('layer-0', {'layer': 'layer-0'})
    producer.flush()
    producer.close()
    
    for layer in layers:
        input_data = layer.forward(input_data)
    redis_client.set('results', json.dumps(int(np.argmax(input_data))))
    return int(np.argmax(input_data))

# Load network
data = load_network("node_based_model.json")
network = build_network(data)

# Load MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])
mnist_test = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

# Test first 10 images one by one
correct_predictions = 0
for i in range(10):
    image, label = mnist_test[i]
    image_np = image.view(-1).numpy()
    redis_client.delete('results')
    prediction = forward_pass(network, image_np)

    print(f"Image {i} Prediction: {prediction}, Label: {label}")
    if prediction == label:
        correct_predictions += 1

accuracy = correct_predictions / 10
print(f"Test Accuracy: {accuracy * 100:.2f}%")


🔥 Sending initial activation message to layer-0...
📤 Layer layer_0 sending activation message to topic layer-0...
✅ Kafka message sent to layer-0: RecordMetadata(topic='layer-0', partition=114, topic_partition=TopicPartition(topic='layer-0', partition=114), offset=0, timestamp=1741375356999, log_start_offset=0, checksum=None, serialized_key_size=-1, serialized_value_size=20, serialized_header_size=-1)
🕒 Neuron 0 in layer_0 waiting for activation on topic layer-0...


KeyboardInterrupt: 

In [1]:
import json
import numpy as np
from torchvision import datasets, transforms
from kafka import KafkaProducer, KafkaConsumer
import redis
import time

# Redis setup
redis_client = redis.Redis(host='host.docker.internal', port=6379, db=0)

# Kafka setup
producer = KafkaProducer(
    bootstrap_servers='kafka:29092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

consumer = KafkaConsumer(
    'activate-layer',
    bootstrap_servers='kafka:29092',
    value_deserializer=lambda m: json.loads(m.decode('utf-8'))
)

try:
    activation_message = {'layer': "layer_0"}
    future = producer.send('activate-layer', activation_message)  # ✅ FIXED
    producer.flush()
    print(f"✅ Kafka message sent to 'activate-layer': {activation_message}")
except Exception as e:
    print(f"❌ Kafka send failed: {e}")
finally:
    producer.close()


✅ Kafka message sent to 'activate-layer': {'layer': 'layer_0'}
